In [1]:
!pip install tqdm

In [2]:
!pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
import urllib.request
from urllib.request import HTTPError
import time
import os
import shutil
from tqdm import tqdm
PATH = './chromedriver'

In [4]:
def download_file(download_url,chapters,filename,manga_name):
    mangaNAME= str('./'+manga_name+'/')
    outdir = mangaNAME
    if not os.path.exists(outdir):
        try:
            os.mkdir(outdir)
        except OSError:
            manga_name=manga_name.rstrip(manga_name[-1])
            mangaNAME= str('./'+manga_name+'/')
            outdir = mangaNAME
            if not os.path.exists(outdir):
                os.mkdir(outdir)
    if not os.path.exists(outdir+chapters):
        os.mkdir(outdir+chapters)
    dirc=outdir+chapters
    fullname = os.path.join(dirc, filename)  
    response = urllib.request.urlopen(download_url) 
    f = open(fullname, 'wb')
    f.write(response.read())
    f.close()
    zip_name= outdir+'/zip/'+chapters
    dir= dirc
    shutil.make_archive(zip_name,'zip',dir)

In [5]:
options = webdriver.ChromeOptions()
options.headless = False # True - the window is hidden, False - the window is not hidden
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=whatever you want")

In [7]:
driver = webdriver.Chrome(PATH,options=options)

In [ ]:
#get website
driver.get("https://www.mangathai.com/")

In [ ]:
#search manga
manganame = input('manganame:')
driver.find_element_by_id('s').send_keys(manganame)
driver.find_element_by_id('s').send_keys(u'\ue007')
time.sleep(3)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'manga-title'))).click()

In [ ]:
manga_name = driver.find_element_by_id('thisPostname').text
print(manga_name)
profile_img=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'img-responsive')))
os.mkdir('./'+manga_name+'/Profile')
os.mkdir('./'+manga_name+'/zip')
file_pro= r'./'+manga_name+'/Profile/profile.png'
profile_img.screenshot(file_pro)
driver.find_element_by_class_name('chapter-name').click()
time.sleep(5)
#click for set display
driver.find_element_by_css_selector('#setup_read > div > div > div:nth-child(1) > label > input[type=radio]').click()
time.sleep(1.5)

In [ ]:
chapter = driver.find_elements_by_tag_name('option')
chapter_page = driver.find_element_by_id('chapter_select').text
chapter_page=chapter_page.split('ตอนที่')
del chapter_page[0]


for i in range(0,len(chapter_page)):
    try:
        chapter_page[i]=int(chapter_page[i])
    except ValueError:
        chapter_page[i]=float(chapter_page[i])  
select = Select(driver.find_element_by_id('chapter_select'))
options = select.options
for index in tqdm(range(0, len(options))):
    chapters ='ตอนที่ '+str(chapter_page[index])
    select.select_by_index(index)
    time.sleep(10)
    select = Select(driver.find_element_by_id('chapter_select'))
    options = select.options
    x = driver.find_elements_by_name('img_page')

    page=1
    for i in x:       
        name=str(page)+'.png'
        try:
            check_same_folder=download_file(i.get_attribute("src"),chapters,name,manga_name)
        except HTTPError:
            pass
        page+=1
        print('[Wait]')              
print('[Finished]')


In [ ]:
#get website
driver.get("https://ballgg.com/wp-login.php?loggedout=true&wp_lang=th")

In [ ]:
#login
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user_login'))).send_keys('adminz')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user_pass'))).send_keys('UTwNlpANpZp@wbFW^w%udLn7')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user_pass'))).send_keys(u'\ue007')

In [ ]:
time.sleep(2)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#menu-posts-wp-manga > a > div.wp-menu-name'))).click()
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'wp-admin-bar-archive'))).click()

In [ ]:
time.sleep(3)
manga_NAME=WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div/div/div[1]/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div/h3/a')))
for i in range(len(manga_NAME)):
    if manga_NAME[i].text == manga_name:
        manga_NAME[i].click()
        break



In [ ]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'wp-admin-bar-edit'))).click()
time.sleep(3)
for i in tqdm(range(len(chapter_page))):
    driver.implicitly_wait(5)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"//*[@id='manga-information-metabox']/div[2]/div[1]/div[1]/ul/li[3]/a"))).click()
    driver.implicitly_wait(5)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#wp-manga-volume > option:nth-child(2)'))).click()
    except TimeoutException:
        pass
    driver.find_element_by_id('wp-manga-chapter-name').send_keys('ตอนที่ '+str(i+1))
    driver.find_element_by_id('upload-zip').click()
    file = './'+manga_name+'/'+'zip/'+'ตอนที่ '+str(i+1)+'.zip'
    driver.find_element_by_id('wp-manga-chapter-file').send_keys(file)
    driver.find_element_by_id('wp-manga-chapter-file-upload').click()
    driver.find_element_by_xpath("//*[@id='publish']").send_keys(Keys.CONTROL + Keys.HOME)
    driver.find_element_by_xpath("//*[@id='publish']").click()
    print('upload ตอนที่'+str(i+1))
    print('Finished')